In [2]:
import pandas as pd
import numpy as np
# import inflect

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys

# reload(sys)
# sys.setdefaultencoding("utf-8")
#TODO: countvectorizer (ignores punctuation), nltk tokenizer

import ExtractHelper as eh
import time

In [31]:
#code to put data from mongodb named Poetry
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY
poems = db.POEM
poemsdf = pd.DataFrame(list(poems.find()))

## some poem videos made it in the data so we need to remove them

In [32]:
poemsdf.columns
coltouse = ['poem_link', 'poet_poems_url','poem_yrpub','poem_title','poem_text']
# poemdf = rawpoem.loc[rawpoem['poem_title'] ][coltouse]

poemsdf = poemsdf.loc[poemsdf['poem_link'].str.contains('video') == False]
poemsdf = poemsdf[coltouse]

In [33]:
# poemsdf.head()

In [34]:
#clean weird unicode stuff
poemsdf['text'] = poemsdf['poem_text'].map(lambda x: [x.encode('ascii','ignore') for x in x])

#compute the feature variables, number of lines, and number of stanzas
poemsdf['numlines'] = poemsdf['text'].apply(lambda x: eh.countlines(x))
poemsdf['numstanzas'] = poemsdf['text'].apply(lambda x: eh.countstanza(x))

In [35]:
#just a quick look
# poemdf.numlines
poemsdf.groupby(['poet_poems_url'])['numstanzas'].mean()[:10]

poet_poems_url
http://www.poets.org/poetsorg/poems/120970    1.000000
http://www.poets.org/poetsorg/poems/120979    1.000000
http://www.poets.org/poetsorg/poems/120986    1.500000
http://www.poets.org/poetsorg/poems/121506    1.000000
http://www.poets.org/poetsorg/poems/123231    2.333333
http://www.poets.org/poetsorg/poems/125236    1.000000
http://www.poets.org/poetsorg/poems/129086    2.666667
http://www.poets.org/poetsorg/poems/339926    1.000000
http://www.poets.org/poetsorg/poems/404586    1.000000
http://www.poets.org/poetsorg/poems/405236    2.000000
Name: numstanzas, dtype: float64

In [36]:
#create feature variable that looks at the average number of lines per stanza for each poem_text
poemsdf['avgline_stanza'] = poemsdf['numlines']/poemsdf['numstanzas']

In [37]:
#create feature variables, wordcount = total number of words, wordcount_d = number of distinct words
#create feature variable ttr = type token ratio as a ratio wordcount_d:wordcount
poemsdf['wordcount'] = poemsdf['text'].apply(lambda x: eh.count_words(x))
poemsdf['wordcount_d'] = poemsdf['text'].apply(lambda x: eh.count_words(x,True))
poemsdf['ttr'] = poemsdf['wordcount_d']/poemsdf['wordcount']

In [14]:
?eh.grab_numerical()

In [38]:
#create feature variable - the year published for each poem
poemsdf['yrpub'] = poemsdf['poem_yrpub'].map(lambda x: eh.grab_numerical(x))

In [39]:
#create a column of just tokenized sentences from the poem text
poemsdf['sentences'] = poemsdf['text'].map(lambda x: eh.sent_tokenize(u' '.join(x)))

In [28]:
#check
# poemsdf.sentences.apply(lambda x: len(x))[4:5]
# print 
# poemsdf.sentences[5]

In [40]:
#create feature variable for the number of sentences in each poem text
poemsdf['num_sentences']= poemsdf['sentences'].map(lambda x: len(x))

In [30]:
?eh.abstraction_score()

In [41]:
#create feature variables, one a regular abstraction score, the other using lesk algorithm for basic word sense
#disambiguation
#Each calculation takes about 10 minutes to run, the following lines of code should take about 20 minutes

#start a timer clock for fun
t0= time.clock()
poemsdf['abstraction_score'] = poemsdf['sentences'].apply(lambda x: eh.abstraction_score(x))
print time.clock() - t0, "seconds process time"

t0= time.clock()
poemsdf['lesk_abs_score'] = poemsdf['sentences'].apply(lambda x: eh.abstraction_score(x, True))
print time.clock() - t0, "seconds process time"

In [45]:
?eh.pronoun_score()

In [42]:
#this might take a while
t0= time.clock()
poemsdf['pronoun_score'] = poemsdf['sentences'].map(lambda x: eh.pronoun_score(x))
print time.clock() - t0, "seconds process time"
t0= time.clock()
poemsdf['pronoun_score'] = poemsdf['pronoun_score']/poemsdf['num_sentences']
print time.clock() - t0, "seconds process time"

In [43]:
poemsdf.columns
# poemdf['bio_sents'] = poemdf['poet_bio'].map(lambda x: sent_tokenize(u''.join(x)))
# poemdf.columns
#looking around to see if anything can be done with the poet bios
# text = poemdf.bio_sents[15]
# text[0]

Index([        u'poem_link',    u'poet_poems_url',        u'poem_yrpub',
              u'poem_title',         u'poem_text',              u'text',
                u'numlines',        u'numstanzas',    u'avgline_stanza',
               u'wordcount',       u'wordcount_d',               u'ttr',
                   u'yrpub',         u'sentences',     u'num_sentences',
       u'abstraction_score',    u'lesk_abs_score',     u'pronoun_score'],
      dtype='object')

In [44]:
# Look at relationship between words in title and words in text?

#abstraction score of title using lesk
#assume each title is a sentence/phrase
poemsdf['title_clean'] = poemsdf['poem_title'].map(lambda x: u''.join(x).replace('[','').replace(']',''))
#just checking if anything can be done
poemsdf['title_lesk_abs'] = poemsdf['title_clean'].map(lambda x: eh.abstraction_score(x, True))

In [45]:
poemsdf.columns

Index([        u'poem_link',    u'poet_poems_url',        u'poem_yrpub',
              u'poem_title',         u'poem_text',              u'text',
                u'numlines',        u'numstanzas',    u'avgline_stanza',
               u'wordcount',       u'wordcount_d',               u'ttr',
                   u'yrpub',         u'sentences',     u'num_sentences',
       u'abstraction_score',    u'lesk_abs_score',     u'pronoun_score',
             u'title_clean',    u'title_lesk_abs'],
      dtype='object')

##CALCULATE PHRASE COMPLEXITY

What is a noun phrase?
e.g. "head of lettuce", "creaky door", "Fred's bagel express"; noun is the head or main word

What is an adjective phrase?
An adjective phrase (or adjectival phrase) is a phrase whose head word is an adjective, e.g. fond of steak, very happy, quite upset about it, etc.

Adverb Adjective
Adjective Preposition Noun
Adverb Adjective Preposition Noun

What is a verb phrase?
A verb phrase is a syntactic unit composed of at least one verb and its dependents—objects, complements and other modifiers—but not always including the subject.

verb noun
verb verb noun

In [46]:
#CREATE FEATURE Conjunction_ratio
t0= time.clock()
poemsdf['conjunction_ratio'] = poemsdf['sentences'].apply(lambda x: eh.conjunction_ratio(x))
print time.clock() - t0, "seconds process time"

In [47]:
# ?eh.naive_enj_score()

t0= time.clock()
poemsdf['enj_score'] = poemsdf['sentences'].apply(lambda x: eh.naive_enj_score(x))
print time.clock() - t0, "seconds process time cacl enjambment score"

#CREATE FEATURES Noun, Adjective, Verb Phrasal Ratios - about 20-25 min each to run

t0= time.clock()
poemsdf['nps_ratio'] = poemsdf['sentences'].apply(lambda x: eh.p_ratio(x,'nps'))
print time.clock() - t0, "seconds process time calc p_ratio"
t0= time.clock()
poemsdf['vps_ratio'] = poemsdf['sentences'].apply(lambda x: eh.p_ratio(x,'vps'))
print time.clock() - t0, "seconds process time calc p_ratio"
t0= time.clock()
poemsdf['aps_ratio']= poemsdf['sentences'].apply(lambda x: eh.p_ratio(x,'aps'))
print time.clock() - t0, "seconds process time calc p_ratio"

#CREATE FEATURES Noun, Adjective, Verb Phrasal Complexity Scores - about 10-15 min to run

t0= time.clock()
poemsdf['avg_nps_cscore'] = poemsdf['sentences'].apply(lambda x: eh.average_phrasal_complexity(x,'nps'))
print time.clock() - t0, "seconds process time"
t0= time.clock()
poemsdf['avg_vps_cscore'] = poemsdf['sentences'].apply(lambda x: eh.average_phrasal_complexity(x,'vps'))
print time.clock() - t0, "seconds process time"
t0= time.clock()
poemsdf['avg_aps_cscore']= poemsdf['sentences'].apply(lambda x: eh.average_phrasal_complexity(x,'aps'))
print time.clock() - t0, "seconds process time"

In [65]:
# poemsdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')
# poemsdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')

In [56]:
extradf = poemsdf[['poem_link', 'sentences']]

In [57]:
t0= time.clock()
extradf['phrasalfreq'] = extradf['sentences'].apply(lambda x: eh.phrasal_freq(x))
print time.clock() - t0, "seconds process time"

In [66]:
#1464
len(extradf)

1464

In [67]:
extradf2 = eh.unpack_dictionary(extradf, 'phrasalfreq', fillna = 0.0)

In [68]:
# extradf2.dtypes
# poemsdf = 
poemsdf = pd.merge(poemsdf, extradf2, how='inner', on=['poem_link'])

In [89]:
# poemsdf.drop('sentences_y', axis = 1, inplace = True)
poemsdf.columns

Index([        u'poem_link',    u'poet_poems_url',        u'poem_yrpub',
              u'poem_title',         u'poem_text',              u'text',
                u'numlines',        u'numstanzas',    u'avgline_stanza',
               u'wordcount',       u'wordcount_d',               u'ttr',
                   u'yrpub',       u'sentences_x',     u'num_sentences',
       u'abstraction_score',    u'lesk_abs_score',     u'pronoun_score',
             u'title_clean',    u'title_lesk_abs', u'conjunction_ratio',
               u'enj_score',         u'nps_ratio',         u'vps_ratio',
               u'aps_ratio',    u'avg_nps_cscore',    u'avg_vps_cscore',
          u'avg_aps_cscore',        u'10w_nps_fr',        u'11w_nps_fr',
              u'12w_nps_fr',        u'13w_nps_fr',        u'14w_nps_fr',
              u'15w_nps_fr',        u'17w_nps_fr',        u'18w_nps_fr',
               u'1w_nps_fr',        u'25w_nps_fr',        u'29w_nps_fr',
               u'2w_nps_fr',        u'34w_nps_fr', 

In [95]:
# poemsdf['78w_nps_fr']

In [93]:
# 29, 34, and 78 are outliers lets take look at the poems in there and drop them
poemsdf = poemsdf.loc[poemsdf['78w_nps_fr']==0] # only one poem has this and its because nltk can't POS correctly
poemsdf = poemsdf.loc[poemsdf['34w_nps_fr']==0] # this is an excerpt of a poem, can drop
poemsdf = poemsdf.loc[poemsdf['29w_nps_fr']==0] # nltk pos def fails on this one, can drop

In [94]:
poemsdf.drop(['78w_nps_fr','34w_nps_fr','29w_nps_fr'], axis = 1, inplace = True)

In [123]:
# poemsdf.columns
poemsdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')
# poemsdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')

#Combine cleaned poet dataframe and poem data frame based on the foreign key identifier, poet_poems_url
poetdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/poetdf.pkl')
resultdf = pd.merge(poetdf, poemsdf, how='inner', on=['poet_poems_url'])

Split tagged poems into groups

1.  Experimental 
    Surrealism – dreamy, used methods like automatic writing (free write), self induced hallucinations, “close observation of the dream state and free play of thought”
    Symbolists – “structures and conceits are built upon grand, illogical, intuitive associations”
    New York School – influenced by surrealism & abstract expressionist art, urban
    Concrete Poetry – visual poems, structure/shape of the poem important
    Language Poetry – language dictates meaning, reader involvement
    Imagism – clarity of expression through precise visual images
    Jazz Poetry – Jazz as poetic inspiration
    Black Mountain – projective verse, remove subjective, project onto the reader
    Conceptual poetry – uncreative, appropriation, the process as poetry

2.  Contemporary, Objectivsts

3.  Modern
    Modernism
    Beat – against conformity, against tradition
    SanFrancisco Renaissance – responding to wars and cultural environment
    Harlem Renaissance – African American identity
    Black arts – civil rights movement, explored African American cultural/historical
    Darkroom Collective – poets of color
    Confessional Poetry – personal, subjective
    Slam/Spokenword – personal subjective

4.  Traditional-Pure
    Formalism, 
    New Formalism – return to traditional forms, meter, rhyme, stanzaic forms
    FiresidePoet – conventional forms, non experimental, attention to rhymes, metrical cadence

In [128]:
resultdf.isnull()
inds = pd.isnull(resultdf).any(1).nonzero()[0]
resultdf.ix[1371]

poet_name                                              Wallace Stevens
poet_url             http://www.poets.org/poetsorg/poet/wallace-ste...
poet_poems_url               http://www.poets.org/poetsorg/poems/45494
tags                                     [MetaphysicalPoet, Modernism]
poet_bio             Wallace Stevens was born in Reading, Pennsylva...
yr_born                                                           1879
yr_died                                                           1955
city                                                           Reading
state                                                               PA
country                                                              A
primary_tag                                           MetaphysicalPoet
tag2                                                         Modernism
tag3                                                               NaN
tag4                                                               NaN
poem_l

In [129]:
resultdf['primary_tag'].value_counts()

# ##lets label by contemporary or not to make this easy on ourselves
resultdf['label'] = resultdf['primary_tag'].map(lambda x: eh.labler(x))
resultdf['strlabel'] = resultdf['primary_tag'].map(lambda x: eh.labler(x, True))


resultdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/resultdf.pkl')
# resultdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/resultdf.pkl')

In [189]:
resultdf= pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/resultdf.pkl')
print resultdf.columns
print len(resultdf)

In [190]:
resultdf = resultdf.query('yrpub >= 1900')

In [191]:
resultdf = resultdf.query('strlabel != ""')
resultdf = resultdf.query('strlabel != "Trad"')

In [192]:
resultdf.strlabel.value_counts()

Contemp    735
Exp        370
Modern     269
dtype: int64

In [193]:
# resultdf.label.value_counts()
resultdf = resultdf.query('primary_tag != "MetaphysicalPoet"')
resultdf.label.value_counts()
len(resultdf)

1374

In [194]:
resultdf.reset_index(drop=True, inplace = True)

In [195]:
resultdf.isnull()
inds = pd.isnull(resultdf).any(1).nonzero()[0]
resultdf.ix[inds]

,poet_name,poet_url,poet_poems_url,tags,poet_bio,yr_born,yr_died,city,state,country,...,2w_nps_fr,3w_nps_fr,4w_nps_fr,5w_nps_fr,6w_nps_fr,7w_nps_fr,8w_nps_fr,9w_nps_fr,label,strlabel
0,Taylor Mali,http://www.poets.org/poetsorg/poet/taylor-mali,http://www.poets.org/poetsorg/poems/406436,[Slam/SpokenWord],Taylor Mali is a four-time National Poetry Sla...,0,0,,,,...,0.181818,0.045455,0.000000,0.000000,0.000000,0.000000,0,0,0,Modern
1,Richard Garcia,http://www.poets.org/poetsorg/poet/richard-garcia,http://www.poets.org/poetsorg/poems/45709,[Contemporary],Richard Garcia is the author of five books of ...,0,0,,,,...,0.291667,0.125000,0.000000,0.000000,0.000000,0.000000,0,0,1,Contemp
2,William Matthews,http://www.poets.org/poetsorg/poet/william-mat...,http://www.poets.org/poetsorg/poems/45636,[Objectivists],"Born in Cincinnati, Ohio, on November 11, 1942...",1942,1997,Cincinnati,,i,...,0.320000,0.080000,0.000000,0.000000,0.000000,0.000000,0,0,1,Contemp
3,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.157895,0.210526,0.052632,0.000000,0.000000,0.000000,0,0,0,Exp
4,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.116279,0.139535,0.023256,0.000000,0.000000,0.000000,0,0,0,Exp
5,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.378378,0.216216,0.000000,0.000000,0.000000,0.000000,0,0,0,Exp
6,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.268293,0.146341,0.000000,0.000000,0.000000,0.000000,0,0,0,Exp
7,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.244898,0.142857,0.020408,0.000000,0.000000,0.000000,0,0,0,Exp
8,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,Exp
9,Philip Appleman,http://www.poets.org/poetsorg/poet/philip-appl...,http://www.poets.org/poetsorg/poems/46510,[Surrealism],"Born in Indiana on February 8, 1926, Philip Ap...",1926,0,,,,...,0.411765,0.151261,0.016807,0.008403,0.000000,0.000000,0,0,0,Exp


In [185]:
# stringcols = ['poet_name','poet_url','poet_poems_url','poet_bio','city','state','country','primary_tag','tags','tag2','tag3','tag4','poem_yrpub','poem_link', 'poem_title', 'title_clean','poem_text', 'sentences_x']
# resultdf.dtypes[20:]
#EXCLUDE wordcount, wordcount_d
# cols = ['yr_born', 'yr_died','yrpub', 'numlines','numstanzas' ,'avgline_stanza','enj_score','ttr',
#              'abstraction_score', 'lesk_abs_score', 'pronoun_score', 'title_lesk_abs', 'conjunction_ratio',
#              'nps_ratio','vps_ratio','aps_ratio','avg_nps_cscore','avg_vps_cscore','avg_aps_cscore',
#              '1w_nps_fr','2w_nps_fr','3w_nps_fr','4w_nps_fr','5w_nps_fr','6w_nps_fr','7w_nps_fr',
#              '8w_nps_fr','9w_nps_fr','10w_nps_fr','11w_nps_fr','12w_nps_fr','13w_nps_fr', '14w_nps_fr',
#              '15w_nps_fr','17w_nps_fr','18w_nps_fr','label', 'strlabel', 'poet_name', 'primary_tag'
#             ]

In [196]:
df = resultdf

In [199]:
df = df.fillna(df.mean())
# strdf = strdf.fillna(strdf.mean())
df

,poet_name,poet_url,poet_poems_url,tags,poet_bio,yr_born,yr_died,city,state,country,...,2w_nps_fr,3w_nps_fr,4w_nps_fr,5w_nps_fr,6w_nps_fr,7w_nps_fr,8w_nps_fr,9w_nps_fr,label,strlabel
0,Taylor Mali,http://www.poets.org/poetsorg/poet/taylor-mali,http://www.poets.org/poetsorg/poems/406436,[Slam/SpokenWord],Taylor Mali is a four-time National Poetry Sla...,0,0,,,,...,0.181818,0.045455,0.000000,0.000000,0.000000,0.000000,0,0,0,Modern
1,Richard Garcia,http://www.poets.org/poetsorg/poet/richard-garcia,http://www.poets.org/poetsorg/poems/45709,[Contemporary],Richard Garcia is the author of five books of ...,0,0,,,,...,0.291667,0.125000,0.000000,0.000000,0.000000,0.000000,0,0,1,Contemp
2,William Matthews,http://www.poets.org/poetsorg/poet/william-mat...,http://www.poets.org/poetsorg/poems/45636,[Objectivists],"Born in Cincinnati, Ohio, on November 11, 1942...",1942,1997,Cincinnati,,i,...,0.320000,0.080000,0.000000,0.000000,0.000000,0.000000,0,0,1,Contemp
3,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.157895,0.210526,0.052632,0.000000,0.000000,0.000000,0,0,0,Exp
4,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.116279,0.139535,0.023256,0.000000,0.000000,0.000000,0,0,0,Exp
5,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.378378,0.216216,0.000000,0.000000,0.000000,0.000000,0,0,0,Exp
6,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.268293,0.146341,0.000000,0.000000,0.000000,0.000000,0,0,0,Exp
7,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.244898,0.142857,0.020408,0.000000,0.000000,0.000000,0,0,0,Exp
8,A. R. Ammons,http://www.poets.org/poetsorg/poet/r-ammons,http://www.poets.org/poetsorg/poems/44389,[Surrealism],Archie Randolph Ammons was born outside Whitev...,1926,2001,,,,...,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,Exp
9,Philip Appleman,http://www.poets.org/poetsorg/poet/philip-appl...,http://www.poets.org/poetsorg/poems/46510,[Surrealism],"Born in Indiana on February 8, 1926, Philip Ap...",1926,0,,,,...,0.411765,0.151261,0.016807,0.008403,0.000000,0.000000,0,0,0,Exp


In [200]:
df.query('yrpub == 0')

# df.describe()
df.groupby(['label'])['yrpub'].median()

label
0    1993
1    2009
Name: yrpub, dtype: int64

In [202]:
df.columns
df.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/clean.pkl')